# SQLite Project
---
```bash
Author: Witchakorn Wanasanwongkot
```
## Chinook database

The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices, and customers. Media-related data was created using real data from an Apple iTunes library.

<p align="center">
    <img src="https://m-soro.github.io/Business-Analytics/SQL-for-Data-Analysis/L4-Project-Query-Music-Store/Misc/001.png" width="100%"/>
</p>

*Image source: https://m-soro.github.io/Business-Analytics/SQL-for-Data-Analysis/L4-Project-Query-Music-Store/Misc/001.png*

In [21]:
import pandas as pd
from sqlite3 import connect

In [22]:
# Connect to the `chinook` database.
conn = connect('.\databases\chinook.db')
print(conn)

## OVERVIEW

In [23]:
# Display the first 5 rows of the table.
query = """
    SELECT * FROM customers LIMIT 5;
"""

pd.read_sql(query, con=conn)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


## SELECT

In [24]:
# Select the data with specific countries and the email contains the domain "@hotmail.com".
query = """
    SELECT
        firstname,
        lastname,
        firstname || ' ' || lastname AS fullname,
        email,
        country
    FROM customers
    WHERE 
        lower(country) IN ('usa', 'brazil', 'united kingdom') AND
        email like '%hotmail%';
"""

pd.read_sql(query, con=conn)

,FirstName,LastName,fullname,Email,Country
0,Kathy,Chase,Kathy Chase,kachase@hotmail.com,USA
1,Richard,Cunningham,Richard Cunningham,ricunningham@hotmail.com,USA
2,Emma,Jones,Emma Jones,emma_jones@hotmail.com,United Kingdom


In [25]:
# Retrieve the data from the table where the customer ID is between 10 and 15, inclusive.
query = """
    SELECT
        firstname,
        lastname,
        firstname || ' ' || lastname AS fullname,
        email,
        country
    FROM customers
    WHERE 
        customerid BETWEEN 10 AND 15;
"""

pd.read_sql(query, con=conn)

,FirstName,LastName,fullname,Email,Country
0,Eduardo,Martins,Eduardo Martins,eduardo@woodstock.com.br,Brazil
1,Alexandre,Rocha,Alexandre Rocha,alero@uol.com.br,Brazil
2,Roberto,Almeida,Roberto Almeida,roberto.almeida@riotur.gov.br,Brazil
3,Fernanda,Ramos,Fernanda Ramos,fernadaramos4@uol.com.br,Brazil
4,Mark,Philips,Mark Philips,mphilips12@shaw.ca,Canada
5,Jennifer,Peterson,Jennifer Peterson,jenniferp@rogers.ca,Canada


In [26]:
# Use regex to select data where the first letter of the first name starts with 'L' or 'A' and the last letter of the country ends with 'e'.
query = """
    SELECT
        firstname,
        lAStname,
        firstname || ' ' || lAStname AS fullname,
        email,
        country
    FROM customers
    WHERE 
        firstname glob '[LA]*' OR country glob '*e';
"""

pd.read_sql(query, con=conn)

,FirstName,LastName,fullname,Email,Country
0,Luís,Gonçalves,Luís Gonçalves,luisg@embraer.com.br,Brazil
1,Leonie,Köhler,Leonie Köhler,leonekohler@surfeu.de,Germany
2,Astrid,Gruber,Astrid Gruber,astrid.gruber@apple.at,Austria
3,Alexandre,Rocha,Alexandre Rocha,alero@uol.com.br,Brazil
4,Aaron,Mitchell,Aaron Mitchell,aaronmitchell@yahoo.ca,Canada
5,Camille,Bernard,Camille Bernard,camille.bernard@yahoo.fr,France
6,Dominique,Lefebvre,Dominique Lefebvre,dominiquelefebvre@gmail.com,France
7,Marc,Dubois,Marc Dubois,marc.dubois@hotmail.com,France
8,Wyatt,Girard,Wyatt Girard,wyatt.girard@yahoo.fr,France
9,Isabelle,Mercier,Isabelle Mercier,isabelle_mercier@apple.fr,France


In [27]:
# Determine the total number of unique countries in the table.
query = """
    SELECT
        count(distinct country) AS country_count
    FROM customers;
"""

pd.read_sql(query, con=conn)

,country_count
0,24


## IF CLAUSE

In [28]:
# Generate a new column named `Region` by mapping the values of the `Country` column to corresponding regions using a CASE statement.
query = """
    SELECT
        country,
        CASE 
            WHEN lower(country) IN ('usa', 'canada') THEN 'America'
            WHEN lower(country) = 'brazil' THEN 'South America'
            WHEN lower(country) IN ('austria', 'belgium', 'denmark') THEN 'Europe'
        ELSE 
            'Other Region' 
        END AS Region
    FROM customers;
"""

pd.read_sql(query, con=conn)

,Country,Region
0,Brazil,South America
1,Germany,Other Region
2,Canada,America
3,Norway,Other Region
4,Czech Republic,Other Region
5,Czech Republic,Other Region
6,Austria,Europe
7,Belgium,Europe
8,Denmark,Europe
9,Brazil,South America


## JOIN

In [29]:
# Retrieve data for tracks with an artist name starting with 'C' and a track time of less than 100000 milliseconds.
query = """
    SELECT
        artists.artistid AS Artist_ID,
        artists.name AS Artist_Name,
        albums.title AS Album_Name,
        tracks.name AS Track_Name,
        tracks.composer AS Track_Composer,
        tracks.milliseconds AS Track_Time
    FROM artists
    LEFT JOIN albums 
        ON artists.artistid = albums.artistid
    LEFT JOIN tracks 
        ON albums.albumid = tracks.albumid
    WHERE 
        artists.name LIKE 'C%' AND milliseconds < 100000
    LIMIT 10;
"""

pd.read_sql(query, con=conn)

,Artist_ID,Artist_Name,Album_Name,Track_Name,Track_Composer,Track_Time
0,16,Caetano Veloso,Prenda Minha,Prenda Minha,Tradicional,99369
1,16,Caetano Veloso,Prenda Minha,"Texto ""Verdade Tropical""",Caetano Veloso,84088
2,18,Chico Science & Nação Zumbi,Afrociberdelia,Mateus Enter,Chico Science,33149
3,18,Chico Science & Nação Zumbi,Afrociberdelia,O Encontro De Isaac Asimov Com Santos Dumont N...,Chico Science,99108
4,18,Chico Science & Nação Zumbi,Afrociberdelia,Enquanto O Mundo Explode,Chico Science,88764
5,18,Chico Science & Nação Zumbi,Afrociberdelia,Interlude Zumbi,Chico Science,71627
6,18,Chico Science & Nação Zumbi,Da Lama Ao Caos,Maracatu De Tiro Certeiro,None,88868
7,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Monteverdi: L'Orfeo,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi,66639


In [30]:
# Combine columns using a WHERE condition.
query = """
    SELECT 
        * 
    FROM artists, albums, tracks 
    WHERE 
        artists.artistid = albums.artistid AND
        albums.albumid = tracks.albumid 
    LIMIT 5;
"""

pd.read_sql(query, con=conn)

,ArtistId,Name,AlbumId,Title,ArtistId,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,AC/DC,1,For Those About To Rock We Salute You,1,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,1,AC/DC,1,For Those About To Rock We Salute You,1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,1,AC/DC,1,For Those About To Rock We Salute You,1,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,1,AC/DC,1,For Those About To Rock We Salute You,1,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,1,AC/DC,1,For Those About To Rock We Salute You,1,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99


## AGGREGATE FUNCTION

In [31]:
# Select the average, sum, minimum, maximum, and count of data grouped by genre.
query = """
    SELECT 
        genres.name AS Genre,
        AVG(bytes) AS avg_bytes, 
        SUM(bytes) AS sum_bytes, 
        MIN(bytes) AS min_bytes, 
        MAX(bytes) AS max_bytes, 
        COUNT(bytes) AS count 
    FROM tracks 
    JOIN genres 
        ON tracks.genreid = genres.genreid 
    WHERE 
        Genre <> 'Latin' 
    GROUP BY Genre 
    HAVING 
        count > 100 
    ORDER BY count DESC;
"""

pd.read_sql(query, con=conn)

,Genre,avg_bytes,sum_bytes,min_bytes,max_bytes,count
0,Rock,9.007374e+06,11682564425,38747,52490554,1297
1,Metal,9.234573e+06,3453730398,1351993,25966720,374
2,Alternative & Punk,7.691003e+06,2553412977,161266,18139840,332
3,Jazz,9.488137e+06,1233457751,4011615,29416781,130


## NESTED QUERY

In [32]:
# Select data from a table with a customer segmentation column and only include rows where the country is "USA".
query = """
    SELECT 
        firstname, 
        lastname, 
        company, 
        coalesce(company, 'End Customer') AS Clean_Company,
        CASE 
            WHEN company IS NULL THEN 'B2B' 
        ELSE 
            'B2C' 
        END AS Segment
    FROM (	
        SELECT 
            * 
        FROM customers 
        WHERE 
            lower(country) = 'usa' 
    );
"""

pd.read_sql(query, con=conn)

,FirstName,LastName,Company,Clean_Company,Segment
0,Frank,Harris,Google Inc.,Google Inc.,B2C
1,Jack,Smith,Microsoft Corporation,Microsoft Corporation,B2C
2,Michelle,Brooks,None,End Customer,B2B
3,Tim,Goyer,Apple Inc.,Apple Inc.,B2C
4,Dan,Miller,None,End Customer,B2B
5,Kathy,Chase,None,End Customer,B2B
6,Heather,Leacock,None,End Customer,B2B
7,John,Gordon,None,End Customer,B2B
8,Frank,Ralston,None,End Customer,B2B
9,Victor,Stevens,None,End Customer,B2B


In [33]:
# Count the total number of customer segments.
query = """
    SELECT  
        Segment, 
        count(*) AS n 
    FROM ( 
        SELECT 
            firstname, 
            lastname, 
            company, 
            coalesce(company, 'End Customer') AS Clean_Company,
            CASE 
                WHEN company IS NULL THEN 'B2B' 
            ELSE 
                'B2C' 
            END AS Segment
        FROM (	
            SELECT 
                * 
            FROM customers 
            WHERE 
                lower(country) = 'usa' 
        ) 
    ) 
    GROUP BY Segment;
"""

pd.read_sql(query, con=conn)

,Segment,n
0,B2B,10
1,B2C,3


## WINDOW FUNCTION

In [34]:
# Generate a running number column partitioning by country.
query = """
    SELECT 
        firstname, 
        lastname, 
        country, 
        ROW_NUMBER() OVER(PARTITION BY country) AS Row_Num 
    FROM customers
    LIMIT 10;
"""

pd.read_sql(query, con=conn)

,FirstName,LastName,Country,Row_Num
0,Diego,Gutiérrez,Argentina,1
1,Mark,Taylor,Australia,1
2,Astrid,Gruber,Austria,1
3,Daan,Peeters,Belgium,1
4,Luís,Gonçalves,Brazil,1
5,Eduardo,Martins,Brazil,2
6,Alexandre,Rocha,Brazil,3
7,Roberto,Almeida,Brazil,4
8,Fernanda,Ramos,Brazil,5
9,François,Tremblay,Canada,1


In [35]:
# Select data where the running number column equals 1.
query = """
    SELECT 
        * 
    FROM ( 
        SELECT 
            firstname, 
            lastname, 
            country, 
            ROW_NUMBER() OVER(PARTITION BY country) AS Row_Num 
        FROM customers 
    ) 
    WHERE 
        Row_Num = 1
    LIMIT 10;
"""

pd.read_sql(query, con=conn)

,firstname,lastname,country,Row_Num
0,Diego,Gutiérrez,Argentina,1
1,Mark,Taylor,Australia,1
2,Astrid,Gruber,Austria,1
3,Daan,Peeters,Belgium,1
4,Luís,Gonçalves,Brazil,1
5,François,Tremblay,Canada,1
6,Luis,Rojas,Chile,1
7,František,Wichterlová,Czech Republic,1
8,Kara,Nielsen,Denmark,1
9,Terhi,Hämäläinen,Finland,1


In [36]:
# Generate a ranking column partitioning by title and sorted by milliseconds.
query = """
    SELECT 
        title, 
        name, 
        milliseconds, 
        RANK() OVER(PARTITION BY title ORDER BY milliseconds DESC) AS Rank 
    FROM tracks 
    JOIN albums 
        ON tracks.albumid = albums.albumid;
"""

pd.read_sql(query, con=conn)

,Title,Name,Milliseconds,Rank
0,...And Justice For All,To Live Is To Die,588564,1
1,...And Justice For All,...And Justice For All,585769,2
2,...And Justice For All,The Frayed Ends Of Sanity,464039,3
3,...And Justice For All,One,446484,4
4,...And Justice For All,Blackened,403382,5
...,...,...,...,...
3498,[1997] Black Light Syndrome,Falling in Circles,549093,3
3499,[1997] Black Light Syndrome,Chaos-Control,529841,4
3500,[1997] Black Light Syndrome,Black Light Syndrome,526471,5
3501,[1997] Black Light Syndrome,Dark Corners,513541,6


In [37]:
# Retrieve the data where the rank is equal to 1.
query = """
    SELECT 
        * 
    FROM (	
        SELECT 
            title, 
            name, 
            milliseconds, 
            RANK() OVER(PARTITION BY title ORDER BY milliseconds DESC) AS Rank 
        FROM tracks 
        JOIN albums ON tracks.albumid = albums.albumid 
    ) 
    WHERE 
        Rank = 1;
"""

pd.read_sql(query, con=conn)

,title,name,milliseconds,Rank
0,...And Justice For All,To Live Is To Die,588564,1
1,20th Century Masters - The Millennium Collecti...,Still Loving You,390674,1
2,"A Copland Celebration, Vol. I",Fanfare for the Common Man,198064,1
3,A Matter of Life and Death,For the Greater Good of God,564893,1
4,A Real Dead One,Hallowed Be Thy Name,471849,1
...,...,...,...,...
342,Warner 25 Anos,O Boto (Bôto),366837,1
343,Weill: The Seven Deadly Sins,Symphony No. 2: III. Allegro vivace,376510,1
344,Worlds,OAM's Blues,266936,1
345,Zooropa,Lemon,418324,1


## SEGMENTATION

In [38]:
# Generate a new segmentation column with 4 NTILE, ordered by milliseconds in descending order.
query = """
    SELECT 
        title, 
        name, 
        milliseconds, 
        NTILE(4) OVER(ORDER BY milliseconds DESC) AS Segment 
    FROM tracks 
    JOIN albums 
        ON tracks.albumid = albums.albumid 
"""

pd.read_sql(query, con=conn)

,Title,Name,Milliseconds,Segment
0,"Battlestar Galactica, Season 3",Occupation / Precipice,5286953,1
1,"Lost, Season 3",Through a Looking Glass,5088838,1
2,"Battlestar Galactica (Classic), Season 1","Greetings from Earth, Pt. 1",2960293,1
3,"Battlestar Galactica (Classic), Season 1",The Man With Nine Lives,2956998,1
4,"Battlestar Galactica (Classic), Season 1","Battlestar Galactica, Pt. 2",2956081,1
...,...,...,...,...
3498,House of Pain,Commercial 1,7941,4
3499,Body Count,Oprah,6635,4
3500,Body Count,A Statistic,6373,4
3501,Body Count,Now Sports,4884,4


In [39]:
# Determine the count of tracks and the average duration (in milliseconds) for each segment.
query = """
    SELECT 
        Segment, 
        COUNT(*) AS Count, 
        ROUND(AVG(milliseconds), 2) AS avg_milliseconds 
    FROM ( 
        SELECT 
            title, 
            name, 
            milliseconds, 
            NTILE(4) OVER(ORDER BY milliseconds DESC) AS Segment 
        FROM tracks 
        JOIN albums 
            ON tracks.albumid = albums.albumid 
    ) 
    GROUP BY Segment;
"""

pd.read_sql(query, con=conn)

,Segment,Count,avg_milliseconds
0,1,876,893267.45
1,2,876,283659.34
2,3,876,231680.26
3,4,875,165529.45
